In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import pandas as pd
import sys
sys.path
sys.path.append('../models')
sys.path.append('../simulation')
import numpy as np
import pickle
import random
import os
import math
import sim_functions_cleaner  as sf
import operator
import study

In [132]:
def get_all_states():
    all_states = []
    counts = {}
    for person,data in experiment.population.items():
        h = data.history
        for time,cd in h.items():
            
            if cd['avail'] and cd['decision_time']:
                states = [1,cd['tod'],cd['dow'],cd['weather'],cd['pretreatment'],cd['location']]
                key = '-'.join(str(i) for i in states)
                if key not in counts: 
                    counts[key]=0
                counts[key]=counts[key]+1
    return counts

In [172]:
coeffs = np.array([ 0.05,  0.25, -0.3 ,  0.25,  0.25, -0.3 ])
#coeffs = np.array([0.05,  0.25,  0.25,  0.25, -0.3 ])

In [134]:
with open('../simulation/visited_states_200.pkl','rb') as f:
    experiment = pickle.load(f)

In [190]:
s = get_all_states()


In [174]:
def get_all_states_by_group(gid):
    all_states = []
    counts = {}
    for person,data in experiment.population.items():
        h = data.history
        
        if data.gid==gid:
            for time,cd in h.items():
            
                if cd['avail'] and cd['decision_time']:
                    states = [1,cd['tod'],cd['dow'],cd['weather'],cd['pretreatment'],cd['location']]
                    key = '-'.join(str(i) for i in states)
                    if key not in counts: 
                        counts[key]=0
                    counts[key]=counts[key]+1
                    all_states.append(states)
    return all_states

In [175]:
group_two = get_all_states_by_group(2)
group_one = get_all_states_by_group(1)

In [176]:
def get_percent_actions(states,coeffs,offset):
    return [(np.dot(coeffs.T,s)+offset)>0 for s in states]

In [177]:
def get_reward(states,coeffs,offset):
    return [(np.dot(coeffs.T,s)+offset) for s in states]

In [197]:
np.array([(np.dot(coeffs.T,st)+0) for st in group_two+group_one]).mean()

0.34394516318729784

In [178]:
per_one = get_percent_actions(group_one,coeffs,0)
per_two = get_percent_actions(group_two,coeffs,0)

In [179]:
sum(per_one)/len(per_one)

0.8289806500569116

In [180]:
sum(per_two)/len(per_two)

0.8007803912947901

In [181]:
rew_one = get_reward(group_one,coeffs,0)
rew_two = get_reward(group_two,coeffs,-.5)
per_one = get_percent_actions(group_one,coeffs,0)
per_two = get_percent_actions(group_two,coeffs,-.5)

In [182]:
sum(per_one)/len(per_one)

0.8289806500569116

In [183]:
sum(per_two)/len(per_two)

0.33430424269070125

In [184]:
for i in [5,25,50,75,95]:
    print(i)
    print(np.percentile(rew_one,i))

5
-0.049999999999999975
25
0.25
50
0.3
75
0.55
95
0.8


In [185]:
for i in [5,25,50,75,95]:
    print(i)
    print(np.percentile(rew_two,i))

5
-0.55
25
-0.3
50
-0.24999999999999994
75
0.050000000000000044
95
0.30000000000000004


In [198]:
-0.24999999999999994+.3

0.050000000000000044

In [199]:
0.050000000000000044/2

0.025000000000000022

In [202]:
np.percentile(rew_one+rew_two,5)

-0.5

In [203]:
.95--.5

1.45

In [204]:
1.45/4

0.3625

In [205]:
for i in [5,25,50,75,95]:
    print(i)
    print(np.percentile(rew_two+rew_one,i))

5
-0.5
25
-0.25
50
0.050000000000000044
75
0.30000000000000004
95
0.8


In [206]:
.8--.5

1.3

In [207]:
1.3/4

0.325